In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [3]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import misc_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
from pathlib import Path
import os
import requests
import ipywidgets as widgets
import glob

### Define Parameters for Data Loading

In [4]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [5]:
if not IN_COLAB:
    # choose local file
    fc = FileChooser()
    fc.default_path = INPUT_FOLDER
    fc.filter_pattern = '*.czi'
    display(fc)

elif IN_COLAB:
    # lislt files inside the folder om gdrive
    czifiles = glob.glob(os.path.join(INPUT_FOLDER, "*.czi"))
    wd = widgets.Select(
        options=czifiles,
        description='CZI Files:',
        layout={'width': 'max-content'}
    )
    display(wd)

FileChooser(path='F:\Github\czitools\data', filename='', title='', show_hidden=False, select_desc='Select', ch…

In [6]:
if not IN_COLAB:
    filepath = fc.selected
elif IN_COLAB:
    filepath = wd.value

print(f"Selected File: {filepath}")

Selected File: F:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small.czi


In [7]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(filepath)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

2024-02-19 14:03:48,640 -  INFO - Reading Dimensions from CZI image data.


SizeS:  None
SizeT:  10
SizeZ:  15
SizeC:  2
SizeY:  256
SizeX:  256


In [8]:
# and get more info
czi_scaling = czimd.CziScaling(filepath)
czi_channels = czimd.CziChannelInfo(filepath)
czi_bbox = czimd.CziBoundingBox(filepath)
czi_objectives = czimd.CziObjectives(filepath)
czi_detectors = czimd.CziDetector(filepath)
czi_microscope = czimd.CziMicroscope(filepath)
czi_sample = czimd.CziSampleInfo(filepath)

2024-02-19 14:03:51,459 -  INFO - Reading Scaling from CZI image data.
2024-02-19 14:03:51,603 -  INFO - Reading Channel Information from CZI image data.
2024-02-19 14:03:51,732 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-19 14:03:51,734 -  INFO - Reading Objective Information from CZI image data.
2024-02-19 14:03:51,876 -  INFO - Reading Detector Information from CZI image data.
2024-02-19 14:03:52,051 -  INFO - Reading Microscope Information from CZI image data.
2024-02-19 14:03:52,176 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-19 14:03:52,305 -  INFO - Reading Dimensions from CZI image data.
2024-02-19 14:03:52,306 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.


get_planetable() got an unexpected keyword argument 'read_one_only'


In [9]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc_tools.md2dataframe(mdict)

# create a ipywdiget to show the dataframe with the metadata
wd1 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd1:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd1]))

2024-02-19 14:03:55,340 -  INFO - Reading Dimensions from CZI image data.
2024-02-19 14:03:55,344 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-19 14:03:55,345 -  INFO - Reading Channel Information from CZI image data.
2024-02-19 14:03:55,346 -  INFO - Reading Scaling from CZI image data.
2024-02-19 14:03:55,346 -  INFO - Reading Objective Information from CZI image data.
2024-02-19 14:03:55,347 -  INFO - Reading Detector Information from CZI image data.
2024-02-19 14:03:55,347 -  INFO - Reading Microscope Information from CZI image data.
2024-02-19 14:03:55,348 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-19 14:03:55,350 -  INFO - Reading Dimensions from CZI image data.
2024-02-19 14:03:55,350 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.
2024-02-19 14:03:55,351 -  INFO - Reading additional Metedata from CZI image data.
2024-02-19 14:03:55,351 -  INFO - No Layers information found.
2024-02-19 1

get_planetable() got an unexpected keyword argument 'read_one_only'


In [10]:
# write XML to disk
xmlfile = czimd.writexml(filepath)
print("XML File written to:", xmlfile)

XML File written to: F:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_CZI_MetaData.xml


In [12]:
# get the planetable for the CZI file
pt = misc_tools.get_planetable(filepath,
                                  norm_time=True,
                                  pt_complete=True,
                                  t=0,
                                  c=0,
                                  z=0)

# create a ipywdiget to show the dataframe with the metadata
wd2 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd2:
    display(HTML(pt.to_html()))
display(widgets.VBox(children=[wd2]))

Reading sublocks planes: 0 2Dplanes [00:00, ? 2Dplanes/s]